In [56]:
import pandas as pd
import plotly.express as px
import os
from IPython.display import display, Markdown

# Dicionários de Estatísticas
offensive_stats = {
    "QB": { "Jardas Passadas": "season_passing_yards", "TDs Passados": "season_pass_touchdown", "Passer Rating": "passer_rating", "Interceptações Sofridas": "season_interception" },
    "RB": { "Jardas Corridas": "season_rushing_yards", "TDs Corridos": "season_rush_touchdown", "Tentativas de Corrida": "season_rush_attempts", "Fumbles": "season_fumble" },
    "WR": { "Jardas Recebidas": "season_receiving_yards", "TDs Recebidos": "season_receiving_touchdown", "Total de Recepções": "season_receptions", "Fumbles": "season_fumble" },
    "TE": { "Jardas Recebidas": "season_receiving_yards", "TDs Recebidos": "season_receiving_touchdown", "Total de Recepções": "season_receptions", "Fumbles": "season_fumble" }
}
defensive_stats = {
    "Interceptações": "interception", "Tackles Solo": "solo_tackle", "Tackles com Assistência": "tackle_with_assist",
    "Sacks": "sack", "QB Hits": "qb_hit", "Fumbles Forçados": "fumble_forced", "Touchdowns Defensivos": "def_touchdown"
}

In [57]:
def load_data():
    """Carrega os dataframes de ataque e defesa."""
    try:
        data_path = "data" 
        offense_file = os.path.join(data_path, "yearly_player_stats_offense.csv")
        defense_file = os.path.join(data_path, "yearly_player_stats_defense.csv")
        
        df_offense = pd.read_csv(offense_file)
        df_defense = pd.read_csv(defense_file)
        
        df_offense['season'] = pd.to_numeric(df_offense['season'])
        df_defense['season'] = pd.to_numeric(df_defense['season'])
        
        return df_offense, df_defense
    except FileNotFoundError:
        print("⚠️ Erro: Arquivos de dados não encontrados. Verifique se a pasta 'data' existe no mesmo diretório do notebook.")
        return pd.DataFrame(), pd.DataFrame()

def aggregate_yearly_stats(df):
    """Agrega estatísticas por jogador e temporada."""
    if df.empty: return pd.DataFrame()
    grouping_cols = ['player_name', 'position', 'season']
    numeric_cols = df.select_dtypes(include='number').columns.drop('season', errors='ignore')
    agg_dict = {col: 'sum' for col in numeric_cols}
    df_agg = df.groupby(grouping_cols, as_index=False).agg(agg_dict)
    return df_agg

# Carrega e processa os dados
df_offense_raw, df_defense_raw = load_data()
df_offense_agg = aggregate_yearly_stats(df_offense_raw)
df_defense_agg = aggregate_yearly_stats(df_defense_raw)

print("Dados carregados e agregados com sucesso!")
display(df_offense_agg.head())

Dados carregados e agregados com sucesso!


,player_name,position,season,birth_year,draft_year,draft_round,draft_pick,draft_ovr,height,weight,...,delta_comp_pct,delta_int_pct,delta_pass_td_pct,delta_ypa,delta_yptarget,delta_ypr,delta_rush_td_pct,delta_ypc,delta_td_pct,delta_yptouch
0,A.J. Barner,TE,2024,2002,2024.0,4.0,21.0,121.0,78.0,251.0,...,0.0,0.0,0.0,0.0,-0.172500,0.021250,0.0,0.000000,0.000000,0.021250
1,A.J. Brown,WR,2019,3994,4038.0,4.0,38.0,102.0,146.0,452.0,...,0.0,0.0,0.0,0.0,0.914599,0.635993,-1.0,1.222222,-1.000000,0.691526
2,A.J. Brown,WR,2020,1997,2019.0,2.0,19.0,51.0,73.0,226.0,...,0.0,0.0,0.0,0.0,0.900735,1.089796,0.0,0.000000,0.000000,1.151261
3,A.J. Brown,WR,2021,3994,4038.0,4.0,38.0,102.0,146.0,452.0,...,-1.0,1.0,-1.0,-1.0,0.473629,1.062804,-1.0,0.165501,1.451923,0.788432
4,A.J. Brown,WR,2022,3994,4038.0,4.0,38.0,102.0,146.0,452.0,...,0.0,0.0,0.0,0.0,0.377826,0.480239,0.0,0.000000,3.416667,0.516481


In [58]:
def plot_top_players_notebook(df, metric, title_metric, season, position=None):
    """Cria e exibe um gráfico de barras com os top 20 jogadores no notebook."""
    if df.empty:
        print("Não há dados para exibir.")
        return

    df_to_plot = df[df["position"] == position] if position else df

    # Filtra por temporada ou total de carreira
    if season != "2012-2024":
        df_filtered = df_to_plot[df_to_plot["season"] == season]
        top20 = df_filtered.nlargest(20, metric)
        title_period = f"na Temporada {season}"
    else:
        career_totals = df_to_plot.groupby(['player_name', 'position'], as_index=False)[metric].sum()
        top20 = career_totals.nlargest(20, metric)
        title_period = "(Total da Carreira, 2012-2024)"

    if top20.empty:
        print("Não foram encontrados dados para os filtros selecionados.")
        return
    
    # Prepara dados para o gráfico
    top20["display_name"] = top20["player_name"] + " (" + top20["position"] + ")"
    top20 = top20.sort_values(by=metric, ascending=True)
    
    # Plota o gráfico
    fig_title = f"Top 20 Jogadores por {title_metric} {title_period}"
    fig = px.bar(
        top20, y="display_name", x=metric, text=metric, title=fig_title,
        labels={"display_name": "Jogador (Posição)", metric: title_metric},
        orientation='h', color=metric, color_continuous_scale="Blues"
    )
    fig.update_traces(texttemplate="%{text:,.0f}", textposition="inside")
    fig.update_layout(title_font_size=24, yaxis={'categoryorder':'total ascending'}, coloraxis_showscale=False, height=800)
    fig.show()

def plot_cumulative_evolution_notebook(df, metric, title_metric, position):
    """Plota a evolução de carreira acumulada dos top 10 jogadores no notebook."""
    df_pos = df[df['position'] == position].copy()
    
    # Encontra os top 10 jogadores da carreira
    career_totals = df_pos.groupby('player_name')[metric].sum().nlargest(10).index
    df_top10 = df_pos[df_pos['player_name'].isin(career_totals)]
    
    # Calcula a soma acumulada
    df_top10 = df_top10.sort_values(['player_name', 'season'])
    df_top10['cumulative_stat'] = df_top10.groupby('player_name')[metric].cumsum()

    # Plota o gráfico
    fig = px.line(
        df_top10, x='season', y='cumulative_stat', color='player_name',
        title=f"Evolução Acumulada de Carreira - Top 10 {position}s por {title_metric}",
        labels={'season': 'Temporada', 'cumulative_stat': f'{title_metric} (Acumulado)', 'player_name': 'Jogador'}
    )
    fig.update_layout(title_font_size=24, height=600)
    fig.show()

In [2]:
display(Markdown("## 📊 Análise de Estatísticas Ofensivas"))

#SELECIONE SEUS FILTROS AQUI
season_filter = 2012 # Opções: "2012-2024" ou um ano específico (ex: 2023)
position_filter = "TE"      # Opções: "QB", "RB", "WR", "TE"
stat_filter = "Jardas Corridas" 
#QB:
# Jardas Passadas: season_passing_yards, TDs Passados: season_pass_touchdown,
# Passer Rating: passer_rating, Interceptações Sofridas: season_interception
#RB:
# Jardas Corridas: season_rushing_yards, TDs Corridos: season_rush_touchdown,
# Tentativas de Corrida: season_rush_attempts, Fumbles: season_fumble
#WR e TE:
# Jardas Recebidas: season_receiving_yards, TDs Recebidos: season_receiving_touchdown,
# Total de Recepções: season_receptions, Fumbles: season_fumble
# ------------------------------------


# Validação e plotagem
if position_filter in offensive_stats and stat_filter in offensive_stats[position_filter]:
    metric_col = offensive_stats[position_filter][stat_filter]
    plot_top_players_notebook(
        df_offense_agg, 
        metric_col, 
        stat_filter, 
        season_filter, 
        position=position_filter
    )
else:
    print("❌ Filtro inválido! Verifique as opções de posição e estatística.")

NameError: name 'Markdown' is not defined

In [63]:
display(Markdown("## 📊 Análise de Estatísticas Defensivas"))

#SELECIONE SEUS FILTROS AQUI
season_filter_def = 2024 # Opções: "2012-2024" ou um ano específico (ex: 2023)
stat_filter_def = "sacks"  
# Interceptações: "interception, Tackles Solo: solo_tackle , Tackles com Assistência : tackle_with_assist
# Sacks: sack, QB Hits: qb_hit ,Fumbles Forçados : fumble_forced ,Touchdowns Defensivos: def_touchdown.

# Validação e plotagem
if stat_filter_def in defensive_stats:
    metric_col_def = defensive_stats[stat_filter_def]
    plot_top_players_notebook(
        df_defense_agg, 
        metric_col_def, 
        stat_filter_def, 
        season_filter_def
    )
else:
    print("❌ Filtro inválido! Verifique as opções de estatística.")

## 📊 Análise de Estatísticas Defensivas

❌ Filtro inválido! Verifique as opções de estatística.


In [26]:
display(Markdown("## 🚀 Evolução Acumulada de Carreira"))

#SELECIONE SEUS FILTROS AQUI
position_evo_filter = "QB" # Opções: "QB", "RB", "WR", "TE"
stat_evo_filter = "Jardas Passadas" # Opções: Veja o dicionário 'offensive_stats'

# Validação e plotagem
if position_evo_filter in offensive_stats and stat_evo_filter in offensive_stats[position_evo_filter]:
    metric_col_evo = offensive_stats[position_evo_filter][stat_evo_filter]
    plot_cumulative_evolution_notebook(
        df_offense_agg, 
        metric_col_evo, 
        stat_evo_filter, 
        position=position_evo_filter
    )
else:
    print("❌ Filtro inválido! Verifique as opções de posição e estatística.")

## 🚀 Evolução Acumulada de Carreira

In [7]:
# Definindo o nome dos arquivos de saída
output_offense_file = "yearly_player_stats_offense_aggregated.csv"
output_defense_file = "yearly_player_stats_defense_aggregated.csv"

# Salvando os arquivos
df_offense_agg.to_csv(output_offense_file, index=False)
df_defense_agg.to_csv(output_defense_file, index=False)

print(f"Arquivos salvos com sucesso:\n- {output_offense_file}\n- {output_defense_file}")

Arquivos salvos com sucesso:
- yearly_player_stats_offense_aggregated.csv
- yearly_player_stats_defense_aggregated.csv
